In [1]:
#!pip install python-constraint2

In [2]:
from dataclasses import dataclass, field
from enum import Enum
from typing import List, Optional, Tuple


Role = Enum('Role', ['MANAGER', 'CASHIER', 'WORKER'])
Status = Enum('Status', ['FULL_TIME', 'PART_TIME'])

@dataclass(frozen=True)
class Person:
    name: str
    role: Role
    availability: int = 0
    status: Status = Status.FULL_TIME
    days: Optional[Tuple[str]] = None
    hours: Optional[Tuple[Tuple[float, float]]] = None
        
@dataclass(frozen=True)
class Shift:
    starting: float
    finishing: float
    unpaid_break: float
    day: str
    paid_time: Optional[float] = field(init=False)
        
    def __post_init__(self):
        if self.starting is not None and self.finishing is not None and self.unpaid_break is not None:
            paid_time = self.finishing - self.starting - self.unpaid_break
            object.__setattr__(self, 'paid_time', paid_time)

        

In [3]:
people: List[Person] = [
    Person("Luciano", Role.MANAGER, status = Status.PART_TIME, availability = 4 ,days=("wednesday", "thursday", "friday", "saturday", "sunday", )), 
    Person("Andrii", Role.MANAGER, availability = 5, days=("monday", "tuesday", "wednesday", "friday", "saturday", "sunday" )), 
    Person("Rikki", Role.CASHIER, availability = 5, days=("monday", "tuesday", "thursday", "friday", "saturday", "sunday" )),
    Person("Rogerio", Role.WORKER, availability = 5, days=("monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday" )),
    Person("Michael", Role.WORKER, ),
    Person("Lucia", Role.WORKER, status = Status.PART_TIME, availability = 5, days=("monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday" ), hours = ((0, 12), (24, 36), (48, 60), (72, 84), (96, 108), (120, 148), (148, 168))),
    Person("Manoel", Role.WORKER, status = Status.PART_TIME, availability = 2, days=("monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday" ), hours = ((12, 24), (36, 48), (60, 72), (84, 96), (108, 120), (120, 148), (148, 168))), 
    Person("Jaemelea", Role.WORKER, status = Status.PART_TIME, availability = 3, days=("tuesday", "wednesday", "thursday", "friday")),
    Person("Gabriely", Role.WORKER, availability = 4, days=("monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday")),
    Person("Felipe", Role.WORKER, availability = 4, days=("monday", "tuesday", "wednesday", "thursday", "friday", "sunday" )),
    Person("Marina", Role.WORKER, availability = 5, days=("monday", "tuesday", "wednesday", "thursday", "friday",)),
    Person("David", Role.WORKER, status = Status.PART_TIME),
    Person("Addis", Role.WORKER, status = Status.PART_TIME, availability = 3, days=("monday", "saturday", "sunday")),
]


In [5]:
import constraint
from typing import List, Tuple, Dict
from collections import Sequence
from itertools import combinations
from constraint import Constraint

#https://github.com/python-constraint/python-constraint/blob/main/constraint/constraints.py
#https://github.com/python-constraint/python-constraint/blob/main/constraint/solvers.py

class EveryDayOfTheWeekConstraint(Constraint):
    def __init__(self, role: Role, nb: int):
        self.role = role
        self.nb = nb

    def __call__(self, variables: Sequence, domains: dict, assignments: dict, forwardcheck=False):        
        condition = set([day for shifts in assignments.values() for day in shifts])
        return condition == {"monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"}
    
class MinNumberOfPeopleDayConstraint(Constraint):
    def __init__(self, nb_people: int, include_weekend: bool):
        self.nb_people = nb_people
        self.weekend = include_weekend
        
    def __call__(self, variables: Sequence, domains: dict, assignments: dict, forwardcheck=False, _unassigned = constraint.Unassigned) -> bool:            
        peeps_per_day = {"monday": 0, "tuesday": 0, "wednesday": 0, "thursday": 0, "friday": 0}
        if self.weekend:
            peeps_per_day.update({"saturday": 0, "sunday": 0})
        
        for key, shifts in assignments.items():              
            peeps_per_day.update({k: peeps_per_day[k] + 1 for k in shifts})
                                   
        return min(peeps_per_day.values()) >= self.nb_people 

class NumberOfPeoplePerDayConstraint(Constraint):
    def __init__(self, nb_people: Dict[str, int]):
        self.nb_people = nb_people
        
    def __call__(self, variables: Sequence, domains: dict, assignments: dict, forwardcheck=False) -> bool:        
        peeps_per_day = {k: 0 for k in self.nb_people.keys()}
        
        for shifts in assignments.values():
            # This is the possible shifts for one person.
            peeps_per_day.update({k: peeps_per_day.get(k, 0) + 1 for k in shifts})
        
        '''
        for k, n in self.nb_people.items():
            if peeps_per_day[k] < n:
                return False
        
        '''
        
        return peeps_per_day == self.nb_people

class EveryHourOfTheDayConstraint(Constraint):
    def __call__(self, variables: Sequence, domains: dict, assignments: dict, forwardcheck=False) -> bool:
        raise NotImplementedError("TODO")
        
class EveryPersonMaxMinHoursConstraint(Constraint):
    def __call__(self, variables: Sequence, domains: dict, assignments: dict, forwardcheck=False) -> bool:
        raise NotImplementedError("TODO")
        
class Org:
    def __init__(self, name: str, staff: List[Person], solver: constraint.Solver = constraint.OptimizedBacktrackingSolver()):
        self.name = name
        self._solver = solver
        self._constraints = None
        self._day_variables, self._hour_variables = self._init_variables(staff)
        self._problem = None

    #initialize domains for variables
    def _init_variables(self, staff) -> Tuple[List[Tuple[str, List]], List[Tuple[str, List]]]:
        days_list = []
        hours_list = []        
        for person in staff:
            days_combinations = (person, [])
            hours_combinations = (person, [])
            
            #add person days combinations from days employee can work those days
            if person.days != None:
                options = []
                if person.status == Status.PART_TIME:
                    for i in range(1, person.availability + 1):
                        options.extend(list(combinations(person.days, i)))
                    days_combinations = (person, options)
                else:
                    days_combinations = (person, list(combinations(person.days, person.availability)))
                
            if person.hours != None:
                hours_combinations = (person, list(combinations(person.hours, person.availability)))
                           
            days_list.append(days_combinations)
            hours_list.append(hours_combinations)
            
        return (days_list, hours_list)
    
    #initialize the problem and add the varibales, domains
    def _init_problem(self, solver: constraint.Solver, variables: List[Tuple[str, List]]) -> constraint.Problem:
        problem = constraint.Problem(solver=solver)
        
        #add the variables and their domains
        for variable in variables:
            if variable[1] != []:
                problem.addVariable(variable[0], variable[1])        
        return problem

    def _print_solution(self, solution: Dict):
        print("======== ROSTER =============")
        if solution is None:
            print("No solution found.")
            return
        for k, v in solution.items():
            print(f"\t-> {k.name} works on {v}")
        print("=============================")

    #add functions to the
    def set_constraints(self, constraints: List[Constraint]):
        self._constraints = constraints
    
    def get_day_roster(self):
        assert self._constraints is not None, "you must call set_constraints() first"
        problem = self._init_problem(self._solver, self._day_variables)       
        
        #adds constraints and variables (if it has domain)
        for constraint in self._constraints:            
            problem.addConstraint(constraint, [variable[0] for variable in self._day_variables if variable[1]])
        
        any_solution = problem.getSolution()        
        self._print_solution(any_solution)

    def get_hour_roster(self):
        assert self._constraints is not None, "you must call set_constraints() first"
        problem = self._init_problem(self._solver, self._hour_variables)
        
        for constraint in self._constraints:            
            problem.addConstraint(constraint, [variable[0] for variable in self._hour_variables if variable[1]])

        any_solution = problem.getSolution()
        self._print_solution(any_solution)


<ipython-input-5-034e199badc5>:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Sequence


In [8]:

org1 = Org("Small Cafe", staff=people[:8], solver = constraint.MinConflictsSolver(steps = 1000)) # First 8 people for this cafe
org2 = Org("Cafe", staff=people, solver = constraint.MinConflictsSolver(steps = 20000)) # IKEA all employees

org1.set_constraints([
    MinNumberOfPeopleDayConstraint(3, True),
    EveryDayOfTheWeekConstraint(role=Role.MANAGER, nb=1)
])

org2.set_constraints([
    NumberOfPeoplePerDayConstraint({"monday":6, "tuesday":7, "wednesday":7, "thursday":7, "friday":6, "saturday":6, "sunday":6}),
    EveryDayOfTheWeekConstraint(role=Role.MANAGER, nb=1),
])

print("-------- Small Cafe ---------")
org1.get_day_roster()


print("-------- Cafe ---------")
org2.get_day_roster()


-------- Small Cafe ---------
======== ROSTER =============
	-> Luciano works on ('saturday', 'sunday')
	-> Andrii works on ('monday', 'wednesday', 'friday', 'saturday', 'sunday')
	-> Rikki works on ('tuesday', 'thursday', 'friday', 'saturday', 'sunday')
	-> Rogerio works on ('monday', 'tuesday', 'wednesday', 'thursday', 'sunday')
	-> Lucia works on ('tuesday', 'wednesday', 'thursday', 'friday', 'saturday')
	-> Manoel works on ('monday', 'tuesday')
	-> Jaemelea works on ('wednesday', 'friday')
-------- Cafe ---------
======== ROSTER =============
	-> Luciano works on ('wednesday', 'thursday', 'saturday', 'sunday')
	-> Andrii works on ('monday', 'tuesday', 'wednesday', 'friday', 'saturday')
	-> Rikki works on ('monday', 'tuesday', 'thursday', 'friday', 'saturday')
	-> Rogerio works on ('tuesday', 'wednesday', 'thursday', 'friday', 'sunday')
	-> Lucia works on ('monday', 'tuesday', 'wednesday', 'thursday', 'sunday')
	-> Manoel works on ('friday', 'saturday')
	-> Jaemelea works on ('tuesd